<a href="https://colab.research.google.com/github/hdgh463/AI/blob/master/HW11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install opencc-python-reimplemented
!wget https://dumps.wikimedia.org/zhwiki/20200301/zhwiki-20200301-pages-articles-multistream1.xml-p1p162886.bz2 #去維基下載資料



from gensim.corpora import WikiCorpus

wiki_corpus = WikiCorpus('zhwiki-20200301-pages-articles-multistream1.xml-p1p162886.bz2', dictionary={}) #放入資料
next(iter(wiki_corpus.get_texts()))[:10] #取1/10

text_num = 0

with open('wiki_text.txt', 'w', encoding='utf-8') as f: #創建文件檔
    for text in wiki_corpus.get_texts(): #加入剛取得1/10的單字
        f.write(' '.join(text)+'\n')
        text_num += 1
        if text_num % 10000 == 0:
            print('{} articles processed.'.format(text_num))

    print('{} articles processed.'.format(text_num))

import jieba
from opencc import OpenCC


# Initial
cc = OpenCC('s2t') #設定轉繁體
train_data = open('wiki_text.txt', 'r', encoding='utf-8').read() #導入資料
train_data = cc.convert(train_data) #轉繁體
train_data = jieba.lcut(train_data) #結巴分詞
train_data = [word for word in train_data if word != ''] #去除空白單字
train_data = ' '.join(train_data) #加入單字
open('seg.txt', 'w', encoding='utf-8').write(train_data) #儲存成文件檔

from gensim.models import word2vec


# Settings
seed = 666 #隨機數
sg = 0 #選擇模型
window_size = 10 #前詞和預測值的最大距離
vector_size = 100 #詞頻大於100丟少的
min_count = 1 #詞頻少於1不用
workers = 8 #一次訓練8個
epochs = 5 #每批中每個單字輪5次
batch_words = 10000 #每一批傳遞的單字量

train_data = word2vec.LineSentence('seg.txt') #放入資料
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    size=vector_size,
    workers=workers,
    iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
) #建立模型

model.save('word2vec.model') #儲存結果


from gensim.models import word2vec

string = '開刀' #輸入
model = word2vec.Word2Vec.load('word2vec.model') #儲存詞向量
print(string)

for item in model.wv.most_similar(string): #找跟輸入有關的詞
    print(item)



     |████████████████████████████████| 491kB 9.5MB/s 
  Created wheel for opencc-python-reimplemented: filename=opencc_python_reimplemented-0.1.5-py2.py3-none-any.whl size=485664 sha256=abed5082c3bba8883c00e6b355fff2a24df5c71b8f7995a96a6d61789a558eff
  Stored in directory: /root/.cache/pip/wheels/36/a0/10/888b9ac7f10154caaa6a73270b1f085e0a7b241baa672bbe49
Successfully built opencc-python-reimplemented
--2020-04-26 06:45:30--  https://dumps.wikimedia.org/zhwiki/20200301/zhwiki-20200301-pages-articles-multistream1.xml-p1p162886.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172586252 (165M) [application/octet-stream]
Saving to: ‘zhwiki-20200301-pages-articles-multistream1.xml-p1p162886.bz2’

zhwiki-20200301-pag 100%[===================>] 164.59M  4.89MB/s    in 34s     

2020-04-26 06:46:04 (4.9

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.717 seconds.
Prefix dict has been built successfully.
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


開刀
('病人', 0.7615276575088501)
('手術', 0.7541905641555786)
('患上', 0.723257839679718)
('不適', 0.719580888748169)
('患有', 0.7112554907798767)
('睡眠', 0.7095690965652466)
('住院', 0.7091484069824219)
('昏迷', 0.7090358734130859)
('病情', 0.703491747379303)
('入院', 0.7003732919692993)


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
